# 1. Import required modules:

In [10]:
import os
import json
import pandas as pd
import time
import requests
from tqdm import tqdm

# 2. Load wallets.csv file

In [13]:
# Load wallet addresses
wallets_df = pd.read_csv('wallets.csv')
wallets_df.columns = ['wallet_id']  # Ensure the column is named correctly
all_wallets = wallets_df['wallet_id'].tolist()

In [15]:
wallets_df.head()

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


# Step-by-Step Working Code with Batch Saving

**1. Set API credentials and constants**

In [18]:
API_KEY = "cqt_rQfCj4pp6XhhX3Cb7CQqxYjkHQwX"  # Replace with your real Covalent API key
CHAIN_ID = 1  # Ethereum Mainnet
BASE_URL = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address"
OUTPUT_DIR = "wallet_data"
WALLETS_PER_FILE = 20

os.makedirs(OUTPUT_DIR, exist_ok=True)


# 2. Define the fetch_transactions function

In [21]:
def fetch_transactions(wallet):
    url = f"{BASE_URL}/{wallet}/transactions_v2/?key={API_KEY}"
    try:
        response = requests.get(url)
        data = response.json()
        return data['data']['items']
    except Exception as e:
        print(f"Error fetching data for {wallet}: {e}")
        return []


# 3. Fetch and save in batches

In [24]:
wallet_transactions = {}
count = 0
file_index = 1

for wallet in tqdm(all_wallets):
    try:
        wallet_transactions[wallet] = fetch_transactions(wallet)
        count += 1
        time.sleep(1)  # Be nice to the API
    except Exception as e:
        print(f"Error for {wallet}: {e}")
        continue

    if count == WALLETS_PER_FILE or wallet == all_wallets[-1]:
        filename = f"wallets_{file_index}.json"
        tmp_file = f"{filename}.tmp"

        try:
            with open(os.path.join(OUTPUT_DIR, tmp_file), "w") as f:
                json.dump(wallet_transactions, f)

            os.replace(
                os.path.join(OUTPUT_DIR, tmp_file),
                os.path.join(OUTPUT_DIR, filename)
            )
            print(f"Saved {filename} with {count} wallets.")
        except Exception as e:
            print(f"Failed to save {filename}: {e}")

        # Reset batch
        wallet_transactions = {}
        count = 0
        file_index += 1


 19%|███████████████▋                                                                 | 20/103 [01:37<14:10, 10.25s/it]

Saved wallets_1.json with 20 wallets.


 39%|███████████████████████████████▍                                                 | 40/103 [02:48<04:44,  4.51s/it]

Saved wallets_2.json with 20 wallets.


 58%|███████████████████████████████████████████████▏                                 | 60/103 [04:25<06:58,  9.74s/it]

...

Saved wallets_4.json with 20 wallets.


 97%|█████████████████████████████████████████████████████████████████████████████▋  | 100/103 [07:44<00:29,  9.97s/it]

Saved wallets_5.json with 20 wallets.


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [07:55<00:00,  4.61s/it]

Saved wallets_6.json with 3 wallets.


# STEP 1: Load Fetched JSON Files into One DataFrame

In [27]:
DATA_DIR = "wallet_data"  # Folder where split JSON files are saved
all_transactions = []

# Load each file and combine into a list
for file in os.listdir(DATA_DIR):
    if file.endswith(".json"):
        with open(os.path.join(DATA_DIR, file), "r") as f:
            wallet_data = json.load(f)
            for wallet, txs in wallet_data.items():
                for tx in txs:
                    tx['wallet'] = wallet  # Add wallet address to each transaction
                    all_transactions.append(tx)

# Convert to DataFrame
df = pd.DataFrame(all_transactions)
print("✅ Total Transactions:", len(df))
df.head()


✅ Total Transactions: 2823


,block_signed_at,block_height,block_hash,tx_hash,tx_offset,successful,miner_address,from_address,from_address_label,to_address,...,gas_metadata,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,pretty_gas_quote,gas_quote_rate,log_events,wallet
0,2025-06-16T21:15:11Z,22719696,0xff57a0c234e73c4897632e198caa0074e84d16080e0f...,0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...,108,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,None,0x13173761e24c3708495b1dd314920f67f97011d0,...,"{'contract_decimals': 18, 'contract_name': 'Et...",84929,55815,4200000000,234423000000000,0.599307,$0.60,2556.520508,"[{'block_signed_at': '2025-06-16T21:15:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,2025-04-27T21:09:35Z,22362988,0xe8a130e1e7110fbdde7dd51e74d940f538057be01240...,0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...,131,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5264347,500000000,2632173500000000,4.722150,$4.72,1794.011719,"[{'block_signed_at': '2025-04-27T21:09:35Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
2,2025-03-24T23:27:47Z,22120041,0x893f17102836ef520b5239cf68595b7c02c78e206582...,0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...,150,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5226097,700000000,3658267900000000,7.601623,$7.60,2077.929443,"[{'block_signed_at': '2025-03-24T23:27:47Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
3,2025-02-23T19:08:59Z,21911013,0x016232bca14eaedba1297086544e1a24a3868ce8d40b...,0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...,107,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xad38352c725c6768c11a9a71bc7d921a9355d56c,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5213387,950000000,4952717650000000,14.001496,$14.00,2827.032959,"[{'block_signed_at': '2025-02-23T19:08:59Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
4,2025-02-11T03:05:11Z,21820480,0x385f706cfcfdf648c9196d4120011c354371bd1977db...,0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...,196,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5244910,1200000000,6293892000000000,16.379029,$16.38,2602.368896,"[{'block_signed_at': '2025-02-11T03:05:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3


# STEP 2: Explore & Clean Transaction Data

In [32]:
# Check available columns
print(df.columns.tolist())

# Make an explicit copy to avoid warnings
df_clean = df[[
    'wallet', 'block_signed_at', 'from_address', 'to_address', 'value', 'gas_spent', 'gas_price', 'successful'
]].copy()

# Fix warning: safely modify the DataFrame
df_clean.loc[:, 'block_signed_at'] = pd.to_datetime(df_clean['block_signed_at'])
df_clean.loc[:, 'value_eth'] = df_clean['value'].astype(float) / 1e18

df_clean.head()


['block_signed_at', 'block_height', 'block_hash', 'tx_hash', 'tx_offset', 'successful', 'miner_address', 'from_address', 'from_address_label', 'to_address', 'to_address_label', 'value', 'value_quote', 'pretty_value_quote', 'gas_metadata', 'gas_offered', 'gas_spent', 'gas_price', 'fees_paid', 'gas_quote', 'pretty_gas_quote', 'gas_quote_rate', 'log_events', 'wallet']


,wallet,block_signed_at,from_address,to_address,value,gas_spent,gas_price,successful,value_eth
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2025-06-16 21:15:11+00:00,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,0x13173761e24c3708495b1dd314920f67f97011d0,0,55815,4200000000,True,0.0
1,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2025-04-27 21:09:35+00:00,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,None,0,5264347,500000000,True,0.0
2,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2025-03-24 23:27:47+00:00,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,None,0,5226097,700000000,True,0.0
3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2025-02-23 19:08:59+00:00,0xad38352c725c6768c11a9a71bc7d921a9355d56c,None,0,5213387,950000000,True,0.0
4,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2025-02-11 03:05:11+00:00,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,None,0,5244910,1200000000,True,0.0


# STEP 3: Feature Engineering Per Wallet

calculate metrics for each wallet to quantify its risk profile.

In [35]:
wallet_features = df_clean.groupby('wallet').agg({
    'value_eth': ['sum', 'mean', 'max'],
    'gas_spent': 'sum',
    'gas_price': 'mean',
    'successful': ['sum', 'count'],
    'block_signed_at': ['min', 'max']
})

# Flatten multi-index columns
wallet_features.columns = ['_'.join(col).strip() for col in wallet_features.columns.values]
wallet_features.reset_index(inplace=True)

# Add active_days feature
wallet_features['active_days'] = (wallet_features['block_signed_at_max'] - wallet_features['block_signed_at_min']).dt.days + 1
wallet_features.drop(columns=['block_signed_at_min', 'block_signed_at_max'], inplace=True)

wallet_features.head()

,wallet,value_eth_sum,value_eth_mean,value_eth_max,gas_spent_sum,gas_price_mean,successful_sum,successful_count,active_days
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,52.254641,0.522546,50.000000,235826553,2.052649e+10,99,100,795
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0.024302,0.004860,0.018600,308732,5.280000e+10,5,5,1
2,0x0795732aacc448030ef374374eaae57d2965c16c,0.018600,0.004650,0.018600,290793,5.850000e+10,4,4,1
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,201.843495,2.523044,67.563152,132280277,3.998171e+10,80,80,2068
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0.026005,0.005201,0.018600,308720,5.214000e+10,5,5,64


# STEP 4: Normalize Features

In [38]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_to_scale = wallet_features.columns.difference(['wallet'])
wallet_features_scaled = wallet_features.copy()
wallet_features_scaled[features_to_scale] = scaler.fit_transform(wallet_features[features_to_scale])
wallet_features_scaled.head()

,wallet,value_eth_sum,value_eth_mean,value_eth_max,gas_spent_sum,gas_price_mean,successful_sum,successful_count,active_days
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0.047611,0.033328,0.100000,0.702143,0.135981,0.989691,1.000000,0.277428
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0.000022,0.000310,0.000037,0.000326,0.359957,0.020619,0.020619,0.000000
2,0x0795732aacc448030ef374374eaae57d2965c16c,0.000017,0.000297,0.000037,0.000273,0.399515,0.010309,0.010309,0.000000
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0.183906,0.160918,0.135126,0.393587,0.270999,0.793814,0.793814,0.722222
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0.000024,0.000332,0.000037,0.000326,0.355377,0.020619,0.020619,0.022013


# STEP 5: Generate Risk Scores (0 to 1000)

In [43]:
# You can use a simple weighted sum model
weights = {
    'value_eth_sum': 0.2,
    'value_eth_mean': 0.1,
    'value_eth_max': 0.1,
    'gas_spent_sum': 0.1,
    'gas_price_mean': 0.1,
    'successful_sum': 0.2,
    'successful_count': 0.1,
    'active_days': 0.1
}

# Compute risk score
wallet_features_scaled['risk_score'] = wallet_features_scaled[list(weights.keys())].mul(
    list(weights.values()), axis=1).sum(axis=1) * 1000
wallet_features_scaled['risk_score'] = wallet_features_scaled['risk_score'].round(0).astype(int)

wallet_features_scaled[['wallet', 'risk_score']].sort_values(by='risk_score', ascending=False).head(10)


,wallet,risk_score
43,0x623af911f493747c216ad389c7805a37019c662d,676
80,0xbd4a00764217c13a246f86db58d74541a0c3972a,629
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,443
96,0xf340b9f2098f80b86fbc5ede586c319473aa11f3,436
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,432
82,0xc22b8e78394ce52e0034609a67ae3c959daa84bc,430
68,0x96479b087cb8f236a5e2dcbfc50ce63b2f421da6,430
9,0x1656f1886c5ab634ac19568cd571bc72f385fdf7,427
30,0x4d997c89bc659a3e8452038a8101161e7e7e53a7,416
13,0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f,415


# STEP 6: Save Final Output

In [46]:
wallet_features_scaled.to_csv("final_wallet_risk_scores.csv", index=False)
print("Risk scores saved to final_wallet_risk_scores.csv")

Risk scores saved to final_wallet_risk_scores.csv
